# 📊 Job Trend Analyzer - Demo Notebook

Dự án phân tích xu hướng thị trường việc làm bằng cách kết hợp **n-gram + embedding + Gemini LLM Agent**, được triển khai chuyên nghiệp theo kiến trúc LangChain.

## 🎯 Mục tiêu
- Phân tích job descriptions từ thị trường việc làm
- Trích xuất các kỹ năng và công nghệ trending
- Gom cụm các kỹ năng tương tự
- Sử dụng AI để phân tích xu hướng và đưa ra insights

## 🔄 Luồng xử lý
```
Job Descriptions → Text Cleaning → N-gram Extraction → Embedding → Clustering → LLM Analysis → Trend Report
```

## ⚙️ Công nghệ sử dụng
- **Text Processing**: NLTK, scikit-learn
- **Embeddings**: Together AI (m2-bert-80M-32k-retrieval)
- **Clustering**: scikit-learn KMeans
- **LLM Agent**: Google Gemini Pro
- **Orchestrator**: LangChain

## 1. 🔧 Setup Environment and Import Libraries

Đầu tiên, chúng ta sẽ cài đặt và import tất cả các thư viện cần thiết.

In [ ]:
# Install required packages (uncomment and run if not installed)
# !pip install together scikit-learn langchain google-generativeai sentence-transformers
# !pip install pandas numpy nltk python-dotenv plotly

import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add project src to path
if '../src' not in sys.path:
    sys.path.append('../src')
if '../' not in sys.path:
    sys.path.append('../')

print("✅ Environment setup completed!")
print(f"Python version: {sys.version}")
print(f"Current working directory: {os.getcwd()}")

In [ ]:
# Import standard libraries
import re
import json
import time
import logging
from typing import List, Dict, Tuple, Optional
from pathlib import Path

# Import data processing libraries
import pandas as pd
import numpy as np

# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
try:
    import plotly.express as px
    import plotly.graph_objects as go
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False
    print("⚠️ Plotly not available. Using matplotlib for visualization.")

# Import ML libraries
try:
    from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
    from sklearn.cluster import KMeans
    from sklearn.decomposition import PCA
    from sklearn.metrics import silhouette_score
    SKLEARN_AVAILABLE = True
    print("✅ Scikit-learn imported successfully")
except ImportError:
    SKLEARN_AVAILABLE = False
    print("❌ Scikit-learn not available")

# Import NLP libraries
try:
    import nltk
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    NLTK_AVAILABLE = True
    print("✅ NLTK imported successfully")
except ImportError:
    NLTK_AVAILABLE = False
    print("❌ NLTK not available")

# Import API clients
try:
    from together import Together
    TOGETHER_AVAILABLE = True
    print("✅ Together AI client imported successfully")
except ImportError:
    TOGETHER_AVAILABLE = False
    print("❌ Together AI client not available")

try:
    import google.generativeai as genai
    GEMINI_AVAILABLE = True
    print("✅ Google Gemini AI imported successfully")
except ImportError:
    GEMINI_AVAILABLE = False
    print("❌ Google Gemini AI not available")

try:
    from langchain.tools import tool
    from langchain_google_genai import ChatGoogleGenerativeAI
    LANGCHAIN_AVAILABLE = True
    print("✅ LangChain imported successfully")
except ImportError:
    LANGCHAIN_AVAILABLE = False
    print("❌ LangChain not available")

print("\n📊 Import Summary:")
print(f"- Scikit-learn: {'✅' if SKLEARN_AVAILABLE else '❌'}")
print(f"- NLTK: {'✅' if NLTK_AVAILABLE else '❌'}")
print(f"- Together AI: {'✅' if TOGETHER_AVAILABLE else '❌'}")
print(f"- Gemini AI: {'✅' if GEMINI_AVAILABLE else '❌'}")
print(f"- LangChain: {'✅' if LANGCHAIN_AVAILABLE else '❌'}")
print(f"- Plotly: {'✅' if PLOTLY_AVAILABLE else '❌'}")

## 2. 🔑 Configure API Keys and Settings

Thiết lập cấu hình cho các API keys và parameters cần thiết.

In [ ]:
# Load environment variables
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    print("⚠️ python-dotenv not available. Set environment variables manually.")

# Configuration class
class Config:
    """Configuration for the Job Trend Analyzer"""
    
    # API Keys (load from environment or set manually)
    TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY", "your_together_api_key_here")
    GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "your_gemini_api_key_here")
    
    # Model configurations
    EMBEDDING_MODEL = "togethercomputer/m2-bert-80M-32k-retrieval"
    LLM_MODEL = "gemini-pro"
    
    # Processing parameters
    NGRAM_RANGE = (1, 3)  # Unigrams to trigrams
    TOP_K_NGRAMS = 50
    N_CLUSTERS = 8
    MIN_WORD_LENGTH = 2
    
    # API parameters
    MAX_RETRIES = 3
    RETRY_DELAY = 1.0
    BATCH_SIZE = 10

# Initialize configuration
config = Config()

# Validate API keys
print("🔍 Checking API Configuration:")
print(f"- Together API Key: {'✅ Set' if config.TOGETHER_API_KEY != 'your_together_api_key_here' else '❌ Not set'}")
print(f"- Gemini API Key: {'✅ Set' if config.GEMINI_API_KEY != 'your_gemini_api_key_here' else '❌ Not set'}")

if config.TOGETHER_API_KEY == "your_together_api_key_here":
    print("\n⚠️ Please set your Together API key:")
    print("   1. Get API key from: https://api.together.xyz/")
    print("   2. Set environment variable: TOGETHER_API_KEY=your_key")
    print("   3. Or update the Config class above")

if config.GEMINI_API_KEY == "your_gemini_api_key_here":
    print("\n⚠️ Please set your Gemini API key:")
    print("   1. Get API key from: https://makersuite.google.com/app/apikey")
    print("   2. Set environment variable: GEMINI_API_KEY=your_key")
    print("   3. Or update the Config class above")

print(f"\n📋 Processing Configuration:")
print(f"- Embedding Model: {config.EMBEDDING_MODEL}")
print(f"- LLM Model: {config.LLM_MODEL}")
print(f"- N-gram Range: {config.NGRAM_RANGE}")
print(f"- Top K N-grams: {config.TOP_K_NGRAMS}")
print(f"- Number of Clusters: {config.N_CLUSTERS}")

## 3. 📝 Text Preprocessing Module

Xây dựng module để làm sạch và xử lý văn bản job descriptions.

In [ ]:
# Download NLTK data if needed
if NLTK_AVAILABLE:
    try:
        nltk.data.find('tokenizers/punkt')
        nltk.data.find('corpora/stopwords')
    except LookupError:
        print("📥 Downloading NLTK data...")
        nltk.download('punkt', quiet=True)
        nltk.download('stopwords', quiet=True)

def clean_text(text: str) -> str:
    """
    Clean and preprocess job description text
    
    Args:
        text: Raw job description text
        
    Returns:
        Cleaned text string
    """
    if not isinstance(text, str):
        return ""
    
    # Convert to lowercase
    text = text.lower().strip()
    
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Remove URLs and email addresses
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    
    # Remove phone numbers
    text = re.sub(r'[\+]?[1-9]?[0-9]{7,15}', '', text)
    
    # Remove special characters but keep letters, numbers, and spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Remove job-specific stopwords
    job_stopwords = {
        'job', 'position', 'role', 'candidate', 'applicant', 'experience',
        'work', 'company', 'team', 'office', 'location', 'salary',
        'benefit', 'requirement', 'qualification', 'responsibility',
        'opportunity', 'career', 'employment', 'hire', 'hiring', 'year', 'years'
    }
    
    # Get English stopwords
    if NLTK_AVAILABLE:
        stop_words = set(stopwords.words('english'))
        stop_words.update(job_stopwords)
        
        # Tokenize and filter
        tokens = word_tokenize(text)
        tokens = [token for token in tokens 
                 if token.lower() not in stop_words 
                 and len(token) >= config.MIN_WORD_LENGTH 
                 and token.isalpha()]
    else:
        # Simple tokenization if NLTK not available
        tokens = text.split()
        tokens = [token for token in tokens 
                 if token.lower() not in job_stopwords 
                 and len(token) >= config.MIN_WORD_LENGTH 
                 and token.isalpha()]
    
    return ' '.join(tokens)

# Test the preprocessing function
test_job_desc = """
We are looking for a Senior Python Developer with 5+ years of experience 
in machine learning and data science. The candidate should have expertise 
in TensorFlow, PyTorch, and scikit-learn. 

Requirements:
- Bachelor's degree in Computer Science
- Experience with AWS/GCP cloud platforms
- Strong knowledge of SQL and NoSQL databases
- Excellent communication skills

Salary: $120,000 - $150,000 per year
Location: San Francisco, CA
Email: jobs@company.com
Phone: (555) 123-4567
Visit our website: https://company.com
"""

print("🧪 Testing Text Preprocessing:")
print("=" * 50)
print("Original text:")
print(test_job_desc[:200] + "...")
print("\n" + "=" * 50)
print("Cleaned text:")
cleaned = clean_text(test_job_desc)
print(cleaned)
print(f"\nLength reduction: {len(test_job_desc)} → {len(cleaned)} characters")

## 4. 🔤 N-gram Extraction Implementation

Triển khai trích xuất n-gram từ văn bản đã được làm sạch.

In [ ]:
def get_ngrams(texts: List[str], ngram_range: Tuple[int, int] = (1, 3), 
               top_k: int = 50, use_tfidf: bool = True) -> List[Tuple[str, float]]:
    """
    Extract n-grams from a list of texts using sklearn
    
    Args:
        texts: List of preprocessed texts
        ngram_range: Range of n-gram sizes (min_n, max_n)
        top_k: Number of top n-grams to return
        use_tfidf: Whether to use TF-IDF or simple count
        
    Returns:
        List of (ngram, score) tuples sorted by score
    """
    if not SKLEARN_AVAILABLE:
        print("❌ Scikit-learn not available. Cannot extract n-grams.")
        return []
    
    if not texts:
        return []
    
    # Filter out empty texts
    texts = [text for text in texts if text.strip()]
    
    if not texts:
        return []
    
    try:
        # Choose vectorizer
        if use_tfidf:
            vectorizer = TfidfVectorizer(
                ngram_range=ngram_range,
                max_features=top_k * 2,  # Get more features for better selection
                min_df=2,  # Must appear in at least 2 documents
                max_df=0.8,  # Must not appear in more than 80% of documents
                lowercase=True,
                token_pattern=r'\b[a-zA-Z][a-zA-Z]+\b'  # Only alphabetic tokens
            )
        else:
            vectorizer = CountVectorizer(
                ngram_range=ngram_range,
                max_features=top_k * 2,
                min_df=2,
                max_df=0.8,
                lowercase=True,
                token_pattern=r'\b[a-zA-Z][a-zA-Z]+\b'
            )
        
        # Fit and transform
        X = vectorizer.fit_transform(texts)
        feature_names = vectorizer.get_feature_names_out()
        
        # Calculate scores (sum across documents)
        scores = X.sum(axis=0).A1
        
        # Create list of (ngram, score) tuples
        ngram_scores = list(zip(feature_names, scores))
        
        # Sort by score (descending) and take top_k
        ngram_scores.sort(key=lambda x: x[1], reverse=True)
        
        return ngram_scores[:top_k]
        
    except Exception as e:
        print(f"❌ Error extracting n-grams: {e}")
        return []

# Test n-gram extraction with sample data
sample_texts = [
    "python developer machine learning tensorflow pytorch",
    "java backend spring boot microservices aws",
    "javascript react frontend angular html css",
    "data scientist python pandas numpy machine learning",
    "devops engineer kubernetes docker aws cloud",
    "fullstack developer python javascript react postgresql",
    "backend engineer java spring boot rest api",
    "ai engineer deep learning tensorflow python",
    "cloud architect aws azure kubernetes microservices",
    "data engineer python sql spark hadoop"
]

# Clean the sample texts
cleaned_texts = [clean_text(text) for text in sample_texts]

print("🧪 Testing N-gram Extraction:")
print("=" * 50)
print(f"Sample texts: {len(sample_texts)}")
print(f"Cleaned texts: {len(cleaned_texts)}")

# Extract n-grams
ngrams = get_ngrams(cleaned_texts, 
                   ngram_range=config.NGRAM_RANGE, 
                   top_k=20, 
                   use_tfidf=True)

print(f"\n📊 Top 20 N-grams (TF-IDF):")
for i, (ngram, score) in enumerate(ngrams):
    print(f"{i+1:2d}. {ngram:<25} (score: {score:.3f})")

# Compare with count-based extraction
ngrams_count = get_ngrams(cleaned_texts, 
                         ngram_range=config.NGRAM_RANGE, 
                         top_k=10, 
                         use_tfidf=False)

print(f"\n📊 Top 10 N-grams (Count):")
for i, (ngram, score) in enumerate(ngrams_count):
    print(f"{i+1:2d}. {ngram:<25} (count: {score:.0f})")

# Analyze n-gram lengths
if ngrams:
    unigrams = [(n, s) for n, s in ngrams if len(n.split()) == 1]
    bigrams = [(n, s) for n, s in ngrams if len(n.split()) == 2] 
    trigrams = [(n, s) for n, s in ngrams if len(n.split()) == 3]
    
    print(f"\n📈 N-gram Distribution:")
    print(f"- Unigrams: {len(unigrams)}")
    print(f"- Bigrams: {len(bigrams)}")
    print(f"- Trigrams: {len(trigrams)}")
    
    if bigrams:
        print(f"\nTop Bigrams: {', '.join([n for n, s in bigrams[:5]])}")
    if trigrams:
        print(f"Top Trigrams: {', '.join([n for n, s in trigrams[:3]])}")

## 5. 🧠 Embedding Generation with Together API

Tạo vector embeddings cho các n-gram sử dụng Together AI API.

In [ ]:
def get_embeddings(phrases: List[str], api_key: str = None) -> List[Tuple[str, List[float]]]:
    """
    Generate embeddings for a list of phrases using Together API
    
    Args:
        phrases: List of phrases to embed
        api_key: Together API key (uses config if not provided)
        
    Returns:
        List of (phrase, embedding) tuples
    """
    if not TOGETHER_AVAILABLE:
        print("❌ Together AI not available. Cannot generate embeddings.")
        return []
    
    api_key = api_key or config.TOGETHER_API_KEY
    
    if api_key == "your_together_api_key_here":
        print("❌ Please set your Together API key first.")
        return []
    
    if not phrases:
        return []
    
    try:
        # Initialize Together client
        client = Together(api_key=api_key)
        
        result = []
        print(f"🔄 Creating embeddings for {len(phrases)} phrases...")
        
        for i, phrase in enumerate(phrases):
            try:
                response = client.embeddings.create(
                    model=config.EMBEDDING_MODEL,
                    input=phrase
                )
                
                if response.data and len(response.data) > 0:
                    embedding = response.data[0].embedding
                    result.append((phrase, embedding))
                
                # Progress indicator
                if (i + 1) % 5 == 0:
                    print(f"  Progress: {i + 1}/{len(phrases)} embeddings created")
                
                # Small delay to be respectful to API
                time.sleep(0.1)
                
            except Exception as e:
                print(f"⚠️ Failed to embed phrase '{phrase}': {e}")
                continue
        
        print(f"✅ Successfully created {len(result)}/{len(phrases)} embeddings")
        return result
        
    except Exception as e:
        print(f"❌ Error creating embeddings: {e}")
        return []

# Create sample embeddings (if API key is available)
if config.TOGETHER_API_KEY != "your_together_api_key_here" and TOGETHER_AVAILABLE:
    print("🧪 Testing Embedding Generation:")
    print("=" * 50)
    
    # Use top 10 n-grams for testing
    test_phrases = [ngram for ngram, score in ngrams[:10]]
    print(f"Test phrases: {test_phrases}")
    
    # Generate embeddings
    embeddings = get_embeddings(test_phrases)
    
    if embeddings:
        print(f"\n📊 Embedding Results:")
        print(f"- Number of embeddings: {len(embeddings)}")
        print(f"- Embedding dimension: {len(embeddings[0][1]) if embeddings else 0}")
        
        # Show first few embeddings (truncated)
        for i, (phrase, embedding) in enumerate(embeddings[:3]):
            embedding_preview = embedding[:5] + ['...'] if len(embedding) > 5 else embedding
            print(f"  {phrase}: {embedding_preview}")
        
        # Basic statistics
        if embeddings:
            all_embeddings = [emb for _, emb in embeddings]
            embedding_matrix = np.array(all_embeddings)
            
            print(f"\n📈 Embedding Statistics:")
            print(f"- Mean magnitude: {np.mean(np.linalg.norm(embedding_matrix, axis=1)):.3f}")
            print(f"- Std magnitude: {np.std(np.linalg.norm(embedding_matrix, axis=1)):.3f}")
            print(f"- Dimension: {embedding_matrix.shape[1]}")
    
else:
    print("⚠️ Skipping embedding generation - API key not available")
    print("Setting up mock embeddings for demonstration...")
    
    # Create mock embeddings for testing without API
    np.random.seed(42)
    test_phrases = [ngram for ngram, score in ngrams[:10]]
    embeddings = []
    
    for phrase in test_phrases:
        # Create mock embedding (768 dimensions like m2-bert)
        mock_embedding = np.random.normal(0, 1, 768).tolist()
        embeddings.append((phrase, mock_embedding))
    
    print(f"✅ Created {len(embeddings)} mock embeddings for testing")
    print(f"   Embedding dimension: {len(embeddings[0][1]) if embeddings else 0}")

# Store embeddings for next steps
embedding_data = embeddings

## 6. 📊 Clustering Implementation

Gom cụm các embeddings để tìm ra các nhóm kỹ năng tương tự.

In [ ]:
def cluster_embeddings(embeddings: List[Tuple[str, List[float]]], 
                      n_clusters: int = 5) -> Dict[int, List[str]]:
    """
    Cluster embeddings using KMeans algorithm
    
    Args:
        embeddings: List of (phrase, embedding) tuples
        n_clusters: Number of clusters to create
        
    Returns:
        Dictionary mapping cluster IDs to lists of phrases
    """
    if not SKLEARN_AVAILABLE:
        print("❌ Scikit-learn not available. Cannot perform clustering.")
        return {}
    
    if not embeddings or len(embeddings) < n_clusters:
        print(f"⚠️ Not enough embeddings ({len(embeddings)}) for {n_clusters} clusters")
        return {}
    
    try:
        # Extract phrases and vectors
        phrases = [phrase for phrase, _ in embeddings]
        vectors = np.array([embedding for _, embedding in embeddings])
        
        print(f"🔄 Clustering {len(phrases)} phrases into {n_clusters} clusters...")
        
        # Perform K-means clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels = kmeans.fit_predict(vectors)
        
        # Organize results into clusters
        clusters = {i: [] for i in range(n_clusters)}
        for phrase, label in zip(phrases, labels):
            clusters[label].append(phrase)
        
        # Calculate clustering quality metrics
        silhouette_avg = silhouette_score(vectors, labels)
        
        print(f"✅ Clustering completed!")
        print(f"   Silhouette Score: {silhouette_avg:.3f}")
        print(f"   Inertia: {kmeans.inertia_:.3f}")
        
        # Display cluster sizes
        cluster_sizes = [len(clusters[i]) for i in range(n_clusters)]
        print(f"   Cluster sizes: {cluster_sizes}")
        
        return clusters
        
    except Exception as e:
        print(f"❌ Error during clustering: {e}")
        return {}

# Perform clustering on the embeddings
if embedding_data:
    print("🧪 Testing Clustering:")
    print("=" * 50)
    
    # Cluster the embeddings
    clusters = cluster_embeddings(embedding_data, n_clusters=min(5, len(embedding_data)))
    
    if clusters:
        print(f"\n📊 Clustering Results:")
        print(f"Number of clusters: {len(clusters)}")
        
        # Display each cluster
        for cluster_id, phrases in clusters.items():
            print(f"\n🏷️ Cluster {cluster_id + 1} ({len(phrases)} items):")
            for phrase in phrases:
                print(f"   • {phrase}")
    
    # Visualize clusters if possible
    if SKLEARN_AVAILABLE and embedding_data and len(embedding_data) > 1:
        print(f"\n📈 Creating 2D visualization...")
        
        # Extract vectors for PCA
        vectors = np.array([embedding for _, embedding in embedding_data])
        phrases = [phrase for phrase, _ in embedding_data]
        
        # Reduce to 2D using PCA
        pca = PCA(n_components=2, random_state=42)
        vectors_2d = pca.fit_transform(vectors)
        
        # Get cluster labels
        if clusters:
            labels = []
            for phrase in phrases:
                for cluster_id, cluster_phrases in clusters.items():
                    if phrase in cluster_phrases:
                        labels.append(cluster_id)
                        break
            
            # Create visualization
            plt.figure(figsize=(12, 8))
            
            # Plot points with cluster colors
            for cluster_id in range(len(clusters)):
                cluster_points = vectors_2d[np.array(labels) == cluster_id]
                if len(cluster_points) > 0:
                    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], 
                              label=f'Cluster {cluster_id + 1}', alpha=0.7, s=100)
            
            # Add labels for points
            for i, phrase in enumerate(phrases):
                plt.annotate(phrase, (vectors_2d[i, 0], vectors_2d[i, 1]), 
                           xytext=(5, 5), textcoords='offset points', 
                           fontsize=8, alpha=0.7)
            
            plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
            plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
            plt.title('2D Visualization of Skill Clusters (PCA)')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
            
            print(f"✅ 2D visualization created")
            print(f"   Total variance explained: {sum(pca.explained_variance_ratio_):.1%}")
    
else:
    print("⚠️ No embeddings available for clustering")
    clusters = {}

## 7. 🤖 LLM Agent Integration with Gemini

Sử dụng Google Gemini để phân tích các cụm và tạo insights về xu hướng.

In [ ]:
def analyze_clusters(clusters: Dict[int, List[str]], api_key: str = None) -> str:
    """
    Analyze clusters using Google Gemini to generate trend insights
    
    Args:
        clusters: Dictionary mapping cluster IDs to lists of phrases
        api_key: Gemini API key (uses config if not provided)
        
    Returns:
        Analysis text from Gemini
    """
    if not GEMINI_AVAILABLE:
        print("❌ Google Gemini AI not available. Cannot perform analysis.")
        return "Gemini AI not available for analysis."
    
    api_key = api_key or config.GEMINI_API_KEY
    
    if api_key == "your_gemini_api_key_here":
        print("❌ Please set your Gemini API key first.")
        return "Gemini API key not configured."
    
    if not clusters:
        return "No clusters available for analysis."
    
    try:
        # Configure Gemini
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel(config.LLM_MODEL)
        
        # Build prompt
        prompt = \"\"\"Bạn là một chuyên gia phân tích thị trường việc làm công nghệ. 
Dưới đây là các nhóm kỹ năng/công nghệ đã được gom cụm từ job descriptions:

\"\"\"\n        
        for cluster_id, phrases in clusters.items():
            prompt += f\"\\nNhóm {cluster_id + 1}: {', '.join(phrases[:10])}\"\n            if len(phrases) > 10:\n                prompt += f\" (và {len(phrases) - 10} kỹ năng khác)\"\n        \n        prompt += \"\"\"\\n\\nHãy phân tích và đưa ra những nhận định sâu sắc về:\\n1. Xu hướng tăng trưởng của từng nhóm kỹ năng\\n2. Những công nghệ/kỹ năng hot nhất hiện tại\\n3. Những kỹ năng đang suy giảm (nếu có)\\n4. Dự đoán xu hướng trong 1-2 năm tới\\n5. Lời khuyên cho người tìm việc trong ngành IT\\n\\nHãy trả lời một cách chi tiết và chuyên nghiệp.\"\"\"\n        \n        print(\"🔄 Analyzing clusters with Gemini AI...\")\n        \n        # Generate response\n        response = model.generate_content(\n            prompt,\n            generation_config={\n                'temperature': 0.3,\n                'max_output_tokens': 1000,\n                'top_p': 0.8,\n                'top_k': 40\n            }\n        )\n        \n        if response.text:\n            print(\"✅ Analysis completed!\")\n            return response.text\n        else:\n            return \"No analysis generated.\"\n            \n    except Exception as e:\n        print(f\"❌ Error during analysis: {e}\")\n        return f\"Error during analysis: {e}\"\n\n# Perform cluster analysis\nif clusters and config.GEMINI_API_KEY != \"your_gemini_api_key_here\" and GEMINI_AVAILABLE:\n    print(\"🧪 Testing LLM Analysis:\")\n    print(\"=\" * 50)\n    \n    analysis_result = analyze_clusters(clusters)\n    \n    print(\"\\n🤖 AI Analysis Results:\")\n    print(\"=\" * 50)\n    print(analysis_result)\n    \nelse:\n    print(\"⚠️ Skipping LLM analysis - API key not available or no clusters\")\n    print(\"Generating mock analysis for demonstration...\")\n    \n    analysis_result = \"\"\"🔍 PHÂN TÍCH XU HƯỚNG THỊ TRƯỜNG VIỆC LÀM IT (Mock Analysis)\n\n📈 XU HƯỚNG TĂNG TRƯỞNG:\n• Nhóm AI/ML: Python, machine learning, deep learning đang có xu hướng tăng mạnh\n• Nhóm Cloud: AWS, Docker, Kubernetes tiếp tục là những kỹ năng hot\n• Nhóm Frontend: React, JavaScript vẫn duy trì sức nóng\n\n🔥 KỸ NĂNG HOT NHẤT:\n1. Python - Ngôn ngữ đa năng, ứng dụng rộng rãi\n2. Machine Learning - Xu hướng AI đang bùng nổ\n3. Cloud Technologies - Chuyển đổi số đẩy nhu cầu cloud\n4. React/JavaScript - Frontend development vẫn rất cần\n\n📉 KỸ NĂNG ĐANG SUY GIẢM:\n• Các công nghệ legacy như VB.NET, Flash\n• Một số framework cũ đang được thay thế\n\n🔮 Dự ĐOÁN 1-2 NĂM TỚI:\n• AI/ML sẽ tiếp tục tăng trưởng mạnh\n• Cloud-native development sẽ trở thành chuẩn\n• Low-code/No-code platforms sẽ phát triển\n• DevOps và automation càng quan trọng\n\n💡 LỜI KHUYÊN:\n1. Tập trung học Python và machine learning\n2. Nắm vững cloud platforms (AWS/Azure)\n3. Phát triển kỹ năng full-stack\n4. Luôn cập nhật công nghệ mới\"\"\"\n    \n    print(\"\\n🤖 Mock Analysis Results:\")\n    print(\"=\" * 50)\n    print(analysis_result)

## 8. ⚙️ LangChain Tool Creation

Tích hợp toàn bộ pipeline vào một LangChain tool để sử dụng dễ dàng.

In [ ]:
if LANGCHAIN_AVAILABLE:\n    @tool\n    def analyze_job_trend(texts: List[str]) -> str:\n        \"\"\"Analyze job market trends from job descriptions.\n        \n        Args:\n            texts: List of job description texts\n            \n        Returns:\n            Comprehensive trend analysis report\n        \"\"\"\n        try:\n            print(\"🚀 Starting Job Trend Analysis Pipeline...\")\n            \n            # Step 1: Clean texts\n            print(\"1️⃣ Cleaning texts...\")\n            cleaned = [clean_text(t) for t in texts if t.strip()]\n            \n            if not cleaned:\n                return \"No valid texts provided for analysis.\"\n            \n            print(f\"   Processed {len(cleaned)} texts\")\n            \n            # Step 2: Extract n-grams\n            print(\"2️⃣ Extracting n-grams...\")\n            ngrams = get_ngrams(cleaned, \n                               ngram_range=config.NGRAM_RANGE, \n                               top_k=config.TOP_K_NGRAMS)\n            \n            if not ngrams:\n                return \"No n-grams could be extracted from the texts.\"\n                \n            print(f\"   Extracted {len(ngrams)} n-grams\")\n            \n            # Step 3: Create embeddings\n            print(\"3️⃣ Creating embeddings...\")\n            phrases = [g[0] for g in ngrams[:20]]  # Limit for demo\n            embeddings = get_embeddings(phrases)\n            \n            if not embeddings:\n                return \"No embeddings could be created. Check API configuration.\"\n                \n            print(f\"   Created {len(embeddings)} embeddings\")\n            \n            # Step 4: Cluster embeddings\n            print(\"4️⃣ Clustering embeddings...\")\n            clusters = cluster_embeddings(embeddings, \n                                        n_clusters=min(config.N_CLUSTERS, len(embeddings)))\n            \n            if not clusters:\n                return \"Clustering failed. Not enough data.\"\n                \n            print(f\"   Created {len(clusters)} clusters\")\n            \n            # Step 5: Analyze with LLM\n            print(\"5️⃣ Analyzing with AI...\")\n            analysis = analyze_clusters(clusters)\n            \n            # Compile final report\n            report = f\"\"\"# 📊 JOB TREND ANALYSIS REPORT\n\n## 📈 Data Summary\n- Job descriptions analyzed: {len(texts)}\n- Valid texts processed: {len(cleaned)}\n- N-grams extracted: {len(ngrams)}\n- Skill clusters identified: {len(clusters)}\n\n## 🔝 Top Trending Skills\n\"\"\"\n            \n            for i, (ngram, score) in enumerate(ngrams[:10]):\n                report += f\"{i+1}. {ngram} (score: {score:.2f})\\n\"\n            \n            report += f\"\\n## 🏷️ Skill Clusters\\n\"\n            for cluster_id, phrases in clusters.items():\n                report += f\"\\n**Cluster {cluster_id + 1}:** {', '.join(phrases[:5])}\"\n                if len(phrases) > 5:\n                    report += f\" (và {len(phrases) - 5} kỹ năng khác)\"\n                report += \"\\n\"\n            \n            report += f\"\\n## 🤖 AI Analysis\\n{analysis}\"\n            \n            print(\"✅ Pipeline completed successfully!\")\n            return report\n            \n        except Exception as e:\n            error_msg = f\"❌ Pipeline failed: {str(e)}\"\n            print(error_msg)\n            return error_msg\n    \n    print(\"✅ LangChain tool 'analyze_job_trend' created successfully!\")\n    print(\"Usage: result = analyze_job_trend.invoke({'texts': your_job_descriptions})\")\n    \nelse:\n    print(\"⚠️ LangChain not available. Tool creation skipped.\")\n    \n    # Create a simple wrapper function instead\n    def analyze_job_trend_simple(texts: List[str]) -> str:\n        \"\"\"Simple version of the job trend analyzer without LangChain\"\"\"\n        try:\n            # Run the pipeline steps\n            cleaned = [clean_text(t) for t in texts if t.strip()]\n            ngrams = get_ngrams(cleaned, ngram_range=config.NGRAM_RANGE, top_k=20)\n            phrases = [g[0] for g in ngrams[:10]]\n            embeddings = get_embeddings(phrases) if phrases else []\n            clusters = cluster_embeddings(embeddings, n_clusters=min(5, len(embeddings))) if embeddings else {}\n            analysis = analyze_clusters(clusters) if clusters else \"No clusters available for analysis.\"\n            \n            # Create simple report\n            report = f\"\"\"JOB TREND ANALYSIS\\n\\nProcessed {len(cleaned)} texts\\nTop skills: {', '.join([n for n, s in ngrams[:5]])}\\n\\nAnalysis:\\n{analysis}\"\"\"\n            \n            return report\n            \n        except Exception as e:\n            return f\"Analysis failed: {str(e)}\"\n    \n    print(\"✅ Simple analyzer function created as fallback\")

## 9. 🔧 Pipeline Integration Testing

Kiểm tra hoạt động của toàn bộ pipeline với dữ liệu mẫu.

In [ ]:
# Test the integrated pipeline with small dataset\ntest_job_descriptions = [\n    \"Senior Python Developer with machine learning experience using TensorFlow and PyTorch\",\n    \"Java Backend Engineer working with Spring Boot and microservices architecture\", \n    \"Frontend Developer specializing in React, Angular, and modern JavaScript frameworks\",\n    \"Data Scientist proficient in Python, pandas, scikit-learn, and statistical analysis\",\n    \"DevOps Engineer experienced with AWS, Kubernetes, Docker, and CI/CD pipelines\"\n]\n\nprint(\"🧪 Testing Complete Pipeline Integration:\")\nprint(\"=\" * 60)\nprint(f\"Test dataset: {len(test_job_descriptions)} job descriptions\")\n\n# Test individual components first\nprint(\"\\n🔍 Component Testing:\")\nprint(\"-\" * 30)\n\n# Test preprocessing\ntest_cleaned = [clean_text(desc) for desc in test_job_descriptions]\nprint(f\"✅ Preprocessing: {len(test_cleaned)} texts cleaned\")\n\n# Test n-gram extraction\ntest_ngrams = get_ngrams(test_cleaned, ngram_range=(1, 2), top_k=15)\nprint(f\"✅ N-gram extraction: {len(test_ngrams)} n-grams extracted\")\n\nif test_ngrams:\n    print(\"   Top 5 n-grams:\", [n for n, s in test_ngrams[:5]])\n\n# Test component integration\nprint(\"\\n🔗 Integration Testing:\")\nprint(\"-\" * 30)\n\nstart_time = time.time()\n\ntry:\n    if LANGCHAIN_AVAILABLE and 'analyze_job_trend' in locals():\n        print(\"Testing LangChain tool...\")\n        result = analyze_job_trend.invoke({\"texts\": test_job_descriptions})\n    else:\n        print(\"Testing simple analyzer...\")\n        result = analyze_job_trend_simple(test_job_descriptions)\n    \n    execution_time = time.time() - start_time\n    \n    print(f\"\\n⏱️ Execution time: {execution_time:.2f} seconds\")\n    print(f\"\\n📋 Pipeline Result:\")\n    print(\"=\" * 60)\n    print(result)\n    \nexcept Exception as e:\n    print(f\"❌ Pipeline test failed: {e}\")\n    print(\"This is expected if API keys are not configured\")\n\n# Performance summary\nprint(\"\\n📊 Performance Summary:\")\nprint(\"=\" * 30)\nprint(f\"- Input texts: {len(test_job_descriptions)}\")\nprint(f\"- Processing time: {execution_time:.2f}s\" if 'execution_time' in locals() else \"- Processing time: N/A\")\nprint(f\"- Average time per text: {(execution_time/len(test_job_descriptions)):.2f}s\" if 'execution_time' in locals() else \"- Average time per text: N/A\")\n\n# Component availability summary\nprint(\"\\n🔧 Component Status:\")\nprint(\"=\" * 30)\ncomponents = {\n    \"Text Preprocessing\": True,\n    \"N-gram Extraction\": SKLEARN_AVAILABLE,\n    \"Embedding Generation\": TOGETHER_AVAILABLE and config.TOGETHER_API_KEY != \"your_together_api_key_here\",\n    \"Clustering\": SKLEARN_AVAILABLE,\n    \"LLM Analysis\": GEMINI_AVAILABLE and config.GEMINI_API_KEY != \"your_gemini_api_key_here\",\n    \"LangChain Integration\": LANGCHAIN_AVAILABLE\n}\n\nfor component, status in components.items():\n    status_icon = \"✅\" if status else \"❌\"\n    print(f\"{status_icon} {component}\")\n\nfunctional_components = sum(components.values())\ntotal_components = len(components)\nprint(f\"\\n📈 Overall Status: {functional_components}/{total_components} components functional ({functional_components/total_components*100:.0f}%)\")"

## 10. 📈 Sample Data Analysis

Chạy phân tích trên bộ dữ liệu lớn hơn để thể hiện khả năng thực tế của hệ thống.

In [ ]:
# Extended sample dataset for comprehensive analysis\nsample_job_dataset = [\n    \"Senior Python Developer with 5+ years experience in machine learning, TensorFlow, PyTorch, and scikit-learn. Knowledge of AWS cloud services required.\",\n    \"Java Backend Engineer to build scalable microservices using Spring Boot, REST APIs, and MySQL. Docker and Kubernetes experience preferred.\",\n    \"Frontend React Developer proficient in JavaScript, TypeScript, Redux, and modern CSS frameworks. Experience with Next.js is a plus.\",\n    \"Data Scientist with expertise in Python, pandas, NumPy, statistical analysis, and machine learning algorithms. SQL and data visualization skills required.\",\n    \"DevOps Engineer experienced with AWS/Azure, Kubernetes, Docker, CI/CD pipelines, Infrastructure as Code, and monitoring tools.\",\n    \"Full Stack Developer using MEAN/MERN stack. Proficiency in Node.js, Express, MongoDB, and Angular/React frameworks required.\",\n    \"AI/ML Engineer to develop deep learning models using TensorFlow, PyTorch, computer vision, and natural language processing techniques.\",\n    \"Cloud Architect to design scalable solutions on AWS/Azure platforms. Experience with serverless computing and microservices architecture.\",\n    \"Mobile Developer creating cross-platform apps with React Native and Flutter. Knowledge of native iOS and Android development preferred.\",\n    \"Cybersecurity Analyst with expertise in penetration testing, vulnerability assessment, network security, and incident response procedures.\",\n    \"QA Engineer skilled in test automation, Selenium, API testing, and performance testing. Experience with Agile methodologies required.\",\n    \"Product Manager with technical background in software development. Experience with user research, data analysis, and product strategy.\",\n    \"Database Administrator proficient in MySQL, PostgreSQL, MongoDB, performance optimization, and backup/recovery procedures.\",\n    \"UI/UX Designer with strong skills in Figma, Adobe Creative Suite, user research, prototyping, and responsive web design principles.\",\n    \"Blockchain Developer experienced with Ethereum, Solidity, smart contracts, DeFi protocols, and Web3 technologies.\"\n]\n\nprint(\"🚀 COMPREHENSIVE JOB TREND ANALYSIS\")\nprint(\"=\" * 60)\nprint(f\"Analyzing {len(sample_job_dataset)} diverse job descriptions...\")\nprint(\"This represents a realistic job market sample.\")\n\n# Run comprehensive analysis\nstart_time = time.time()\n\ntry:\n    # Choose the appropriate analyzer\n    if LANGCHAIN_AVAILABLE and 'analyze_job_trend' in locals():\n        print(\"\\n🔧 Using LangChain-powered analyzer...\")\n        final_result = analyze_job_trend.invoke({\"texts\": sample_job_dataset})\n    else:\n        print(\"\\n🔧 Using simple analyzer...\")\n        final_result = analyze_job_trend_simple(sample_job_dataset)\n    \n    total_time = time.time() - start_time\n    \n    print(f\"\\n\" + \"=\" * 80)\n    print(\"📊 FINAL ANALYSIS RESULTS\")\n    print(\"=\" * 80)\n    print(final_result)\n    \n    print(f\"\\n\" + \"=\" * 80)\n    print(\"⏱️ PERFORMANCE METRICS\")\n    print(\"=\" * 80)\n    print(f\"• Total processing time: {total_time:.2f} seconds\")\n    print(f\"• Average time per job description: {total_time/len(sample_job_dataset):.2f} seconds\")\n    print(f\"• Dataset size: {len(sample_job_dataset)} job descriptions\")\n    print(f\"• Text processing rate: {len(sample_job_dataset)/total_time:.1f} jobs/second\")\n    \nexcept Exception as e:\n    print(f\"\\n❌ Comprehensive analysis failed: {e}\")\n    print(\"\\n🔍 Likely causes:\")\n    print(\"  - API keys not configured correctly\")\n    print(\"  - Missing dependencies\")\n    print(\"  - Network connectivity issues\")\n    print(\"  - API rate limits exceeded\")\n\n# Generate summary report\nprint(f\"\\n\" + \"=\" * 80)\nprint(\"📋 PROJECT SUMMARY\")\nprint(\"=\" * 80)\nprint(\"\"\"\n🎯 JOB TREND ANALYZER - COMPLETE IMPLEMENTATION\n\n✅ IMPLEMENTED FEATURES:\n• Text preprocessing and cleaning\n• N-gram extraction with TF-IDF scoring\n• Embedding generation using Together AI (m2-bert)\n• K-means clustering for skill grouping\n• LLM-powered analysis using Google Gemini\n• LangChain tool integration\n• End-to-end pipeline automation\n\n🔧 TECHNICAL COMPONENTS:\n• Modular architecture with clear separation of concerns\n• Error handling and fallback mechanisms\n• Performance monitoring and metrics\n• Scalable design for larger datasets\n\n📊 ANALYSIS CAPABILITIES:\n• Skill trend identification\n• Technology cluster discovery\n• Market insight generation\n• Career recommendation system\n• Automated report generation\n\n🚀 NEXT STEPS:\n1. Deploy as a web application using Streamlit\n2. Add real-time job scraping capabilities\n3. Implement time-series trend analysis\n4. Create interactive visualizations\n5. Build API endpoints for integration\n6. Add more sophisticated NLP models\n7. Implement caching for better performance\n\n💡 USAGE:\n- For job seekers: Identify trending skills and career paths\n- For employers: Understand market demands and skill gaps\n- For educators: Align curriculum with industry needs\n- For researchers: Analyze job market evolution\n\"\"\")\n\nprint(f\"\\n🎉 Demo completed successfully!\")\nprint(f\"📚 Check the generated reports and visualizations above.\")\nprint(f\"🔗 All components are now ready for production use.\")